In [54]:
import os
import pandas as pd
import requests
from dotenv import load_dotenv
from urllib.parse import urlencode

# Load environment variables
load_dotenv()

# API key from .env file
API_KEY = os.getenv('DISTANCE_MATRIX_API_KEY')

# Load the geocoded data
walmart_df = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/interim/walmart_sample_geocoded.csv')
zips_df = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/interim/zips_sample_geocoded.csv')

# Extract the state from the Walmart addresses and add it as a new column
walmart_df['State'] = walmart_df['Address'].apply(lambda x: x.split(',')[-1].strip())

# Output Files (CSV)
output_file = '/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/interim/distance_sample.csv'

In [55]:
walmart_df

,Address,Designation,Store #,Open Date,latitude,longitude,State
0,"1151 Stonecrest Blvd., Tega Cay, South Carolina",Walmart Supercenter,3733,"March 7, 2008",35.046191,-80.994007,South Carolina
1,"2150 E Tangerine Rd., Oro Valley, Arizona",Walmart Supercenter,3379,"October 29, 2008",32.426144,-110.939906,Arizona
2,"3200 Lusk Dr., Neosho, Missouri",Walmart Supercenter,17,"May 1, 1969",36.841710,-94.394270,Missouri
3,"10300 E Highway 350, Raytown, Missouri",Walmart Supercenter,1094,"October 1, 1987",38.984768,-94.460438,Missouri
4,"4517 N Midland Dr., Midland, Texas",Walmart Supercenter,608,"June 29, 1984",32.029424,-102.145934,Texas
...,...,...,...,...,...,...,...
375,"5219 Highway 51 N, Senatobia, Mississippi",Walmart Supercenter,155,"August 1, 1977",34.575831,-89.968427,Mississippi
376,"2550 S Kolb Rd., Tucson, Arizona",Neighborhood Market,3049,"April 19, 2013",32.192946,-110.842481,Arizona
377,"2121 N Collins St., Arlington, Texas",Neighborhood Market,5613,"January 16, 2013",32.768159,-97.095910,Texas
378,"2586 N Slappey Blvd., Albany, Georgia",Neighborhood Market,4517,"January 20, 2016",31.613019,-84.174321,Georgia


In [56]:
zips_df

,zip,ruca,city,state_name,population,county,median_household_income,stratum_size,sample_proportion,stratum_sample_size,latitude,longitude
0,35575,Rural,Lynn,Alabama,"1,020",Winston,"$43,289",82,0.002438,1,34.049571,-87.552646
1,36866,Suburban,Notasulga,Alabama,"4,145",Macon,"$52,123",365,0.010852,4,32.536635,-85.697354
2,36026,Suburban,Equality,Alabama,"1,239",Coosa,"$89,375",365,0.010852,4,32.775220,-86.121600
3,35972,Suburban,Gallant,Alabama,"1,036",Etowah,NaN,365,0.010852,4,34.002330,-86.224672
4,35987,Suburban,Steele,Alabama,"2,483",St. Clair,"$62,703",365,0.010852,4,33.916778,-86.243593
...,...,...,...,...,...,...,...,...,...,...,...,...
351,53955,Suburban,Poynette,Wisconsin,"5,662",Columbia,"$84,151",360,0.010704,4,43.433273,-89.394518
352,54603,Urban,La Crosse,Wisconsin,"14,198",La Crosse,"$48,580",198,0.005887,2,43.853763,-91.246238
353,53127,Urban,Genesee Depot,Wisconsin,108,Waukesha,"$132,950",198,0.005887,2,42.960098,-88.374455
354,82932,Rural,Farson,Wyoming,705,Sweetwater,"$63,520",88,0.002616,1,42.083362,-109.418358


In [57]:
def fetch_and_append_distances(state, output_file):
    # Filter dataframes for the state
    state_walmart_df = walmart_df[walmart_df['State'] == state]
    state_zips_df = zips_df[zips_df['state_name'] == state]

    for _, zip_row in state_zips_df.iterrows():
        origin = f"{zip_row.latitude},{zip_row.longitude}"
        for _, walmart_row in state_walmart_df.iterrows():
            destination = f"{walmart_row.latitude},{walmart_row.longitude}"
            
            # API request
            url = f"https://api.distancematrix.ai/maps/api/distancematrix/json?origins={origin}&destinations={destination}&key={API_KEY}"
            response = requests.get(url)
            
            # Initialize the dictionary that will hold our row data
            row_data = {
                'origin': origin,
                'destination': destination,
                'distance_in_time': 'N/A'  # Default value in case of an error
            }

            if response.status_code == 200:
                data = response.json()
                if data['status'] == 'OK':
                    distance_info = data['rows'][0]['elements'][0]
                    if distance_info['status'] == 'OK':
                        row_data['distance_in_time'] = distance_info['duration']['text']
                        
            # Append the result to the CSV file
            with open(output_file, 'a') as f:
                pd.DataFrame([row_data]).to_csv(f, header=False, index=False)

# Iterate through each state and append results to the CSV
for state in zips_df['state_name'].unique():
    fetch_and_append_distances(state, output_file)

print("Distance data appended to CSV successfully.")

Distance data appended to CSV successfully.
